In [1]:
from pyspark.sql import SparkSession
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

# 创建 SparkSession
spark = SparkSession.builder \
    .appName("PySpark Dataloader") \
    .getOrCreate()

# 读取数据
df_spark = spark.read.csv("test_pyspark.csv", header=True, inferSchema=True)

# 假设我们需要这些特征作为模型输入：feature1, feature2, ... , featureN
# 和目标变量：target
df_spark = df_spark.select("feature1", "feature2", "featureN", "target")

# 展示预处理后的数据
df_spark.show(5)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/08 21:06:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-----------+----------+----------+------------+
|   feature1|  feature2|  featureN|      target|
+-----------+----------+----------+------------+
| -1.1951252|0.71497804|0.67314315| -0.18863806|
|-0.41465077|-0.3884412| 0.3669275|-0.097634405|
| -2.1843934|-1.3371152|0.56692296|  -0.5408166|
|  2.6698067|0.83432883|-0.4314149| -0.68673754|
| -1.0006564|  2.007322| 0.5546377|   1.1798972|
+-----------+----------+----------+------------+
only showing top 5 rows



In [2]:
sales_df = spark.read.csv( "daily_sales_*.csv", header=True, inferSchema=True) 

In [3]:
sales_df.head(n=10)

[Row(order_id=1001, customer_id=123, name='John Doe', age=35, phone='555-123-4567', region='North', amount='899.50', department='IT'),
 Row(order_id=1002, customer_id=456, name='Jane Smith', age=28, phone=None, region=' "South"', amount='2450.00', department='HR'),
 Row(order_id=1003, customer_id=789, name='Bob Johnson', age=42, phone='555-987-6543', region='East', amount='320.25', department='IT'),
 Row(order_id=1004, customer_id=101, name='Alice Brown', age=23, phone='555-555-5555', region='West', amount='null', department='Finance'),
 Row(order_id=1005, customer_id=202, name='Charlie Lee', age=50, phone=None, region=' "North"', amount='1750.99', department='HR'),
 Row(order_id=1006, customer_id=303, name='Diana King', age=31, phone='555-111-2222', region='South', amount='null', department='IT')]

In [4]:
from pyspark.sql import functions as F
sales_df.filter(F.isnull(sales_df['customer_id']))

DataFrame[order_id: int, customer_id: int, name: string, age: int, phone: string, region: string, amount: string, department: string]

In [5]:
cleaned_df = sales_df.dropDuplicates(["order_id"]).fillna({"customer_id":0, "amount":0.0})

In [6]:
cleaned_df

DataFrame[order_id: int, customer_id: int, name: string, age: int, phone: string, region: string, amount: string, department: string]

In [7]:
from pyspark.sql.functions import when, sum, col, count

transformed_df = cleaned_df.withColumn("sales_category", 
                    when(col("amount") > 1000, "high" ). 
                    when(col("amount") > 500, "medium"). 
                    otherwise("low")
                ).groupBy("region", "sales_category").agg(
                    sum("amount").alias( "total_sales"), 
                    count("*").alias("order_count")
                )

25/03/08 22:41:50 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1044426 ms exceeds timeout 120000 ms
25/03/08 22:41:50 WARN SparkContext: Killing executors is not supported by current scheduler.
25/03/08 22:41:51 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$

In [ ]:
# 5. 写入 Hive 分区表
transformed_df.write \
    .partitionBy("region") \
    .format("parquet") \
    .saveAsTable("regional_sales")

In [ ]:
# 6. 记录元数据（可选
spark.sql("ANALYZE TABLE regional_sales COMPUTE STATISTICS")

In [ ]:
transformed_df.columns

In [ ]:
df = spark.read.csv ( "salary.csv", header=True, inferSchema=True, escape='"')

In [ ]:
df.head(n=10)

In [ ]:
# 示例1: 列操作 + 过滤 
from pyspark.sql.functions import col, upper 

transformed_df = df.select(
    upper(col("name")).alias("name_upper"), 
    col("age").cast("integer"), 
    (col("salary") * 1.1).alias("adjusted_salary")
).filter(col("age") > 18 )

In [ ]:
transformed_df.head(n=10)

In [ ]:
# 示例2: 聚合操作
agg_df = df.groupBy( "department") \
    .agg({"salary" : "avg", "id" : "count"}) \
    .withColumnRenamed("avg(salary)", "avg_salary")

In [ ]:
agg_df.head(n=10)

In [ ]:
import numpy as np
import pandas as pd

data = np.random.randn(100, 4)
df = pd.DataFrame(data, dtype=np.float32, columns=['feature1', 'feature2', 'featureN', 'target'])
df.to_csv('test_pyspark.csv')

In [ ]:
# 将 Spark DataFrame 转换为 Pandas DataFrame
#df_pandas = df_spark.toPandas()
df_pandas = df

# 将特征和标签分别提取
X = df_pandas[["feature1", "feature2", "featureN"]].values
y = df_pandas["target"].values

# 将特征和标签转换为 PyTorch 张量
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# 创建数据集对象
dataset = CustomDataset(X_tensor, y_tensor)

In [ ]:
# 定义 DataLoader
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# 打印批次数据
for batch_X, batch_y in dataloader:
    print(batch_X.shape, batch_y.shape)


In [ ]:
import torch.nn as nn
import torch.optim as optim

# 定义一个简单的模型
class SimpleModel(nn.Module):
    def __init__(self, input_dim):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(input_dim, 1)
    
    def forward(self, x):
        return self.fc(x)

# 实例化模型
model = SimpleModel(input_dim=X_tensor.shape[1])

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 10
for epoch in range(num_epochs):
    for batch_X, batch_y in dataloader:
        print(batch_X.shape, batch_y.shape)
        # 前向传播
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y.unsqueeze(1))
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
